In [496]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import utils as ut
import LIM_class
import torch
import torch.nn as nn
import torch.optim as optim
import json
import itertools
from torch.utils.data import DataLoader, Dataset

plt.style.use("../plotting.mplstyle")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [497]:
data = xr.open_dataset("./data/ts_Amon_CESM2_piControl_r1i1p1f1.nc")["ts"]
#data = xr.open_dataset("./data/zos_Amon_CESM2_piControl_r1i1p1f1.nc")["zos"]
#data_old = xr.open_dataset("./data/ssta_1950_2021.nc")["ssta"]
mask = xr.open_dataset("./data/sftlf_fx_CESM2_historical_r1i1p1f1.nc")["sftlf"]

#14400 orginial size
data = data[:1000, :, :]

data = ut.apply_mask(mask, data)
#print("Data : {} + shape {}".format(data, data.shape))

data_anomalies = ut.calculate_monthly_anomalies(data)
#print("Month mean : {} + shape : {}".format(data_anomalies, data_anomalies.shape))

data_cropped =ut.crop_xarray(data_anomalies)
#print("Data cropped : {} + shape : {}".format(data_cropped, data_cropped.shape))


pca_10 = ut.SpatioTemporalPCA(data_cropped, n_components=20)
#pca_10 = ut.SpatioTemporalPCA(data_anomalies, n_components=20)
eof_10 = pca_10.eofs()
pc_10 = pca_10.principal_components()

In [498]:
# Create training and test data
data = pca_10.principal_components()
index_train = int(0.8 * len(data["time"]))
data_train = data[:, :index_train]
data_test = data[:, index_train:]

# Creating an example LIM object
tau = 1
model = LIM_class.LIM(tau)
model.fit(data_train.data)
#print("Data train : {} + shape: {} + type: {}".format(data_train.data[:5], data_train.data.shape, type(data_train.data)))

In [ ]:
from FNN_model import *

# Set random seed for reproducibility
torch.manual_seed(42)

def hyperparameter_training_loop(dataloader, input_size, output_size):

    # Generate all possible combinations of hyperparameters
    combinations = list(itertools.product(*hyperparams.values()))

    overall_best_model = {"hidden_size": None,
                         "learning_rate": None,
                         "num_epochs": None,
                         "loss": float('inf')}

    # Iterate over each hyperparameter combination
    for i, params in enumerate(combinations):
        print(f"Testing parameter combination {i+1}/{len(combinations)}: {params}")

        hidden_size = params[0]
        learning_rate = params[1]
        num_epochs = params[2]

        # Train the model
        model = FeedforwardNetwork(input_size, hidden_size, output_size)
        losses = train(model, dataloader, num_epochs, learning_rate, input_size)

        # Save the model and hyperparameters to a file
        result = {
            'hyperparameters': {
                'hidden_size': params[0],
                'learning_rate': params[1],
                'num_epochs': params[2],
                'best_loss': losses[-1],
                "losses": losses,
            }
        }
        if losses[-1] < overall_best_model["loss"]:
            overall_best_model["hidden_size"] = params[0]
            overall_best_model["learning_rate"] = params[1]
            overall_best_model["num_epochs"] = params[2]
            overall_best_model["loss"] = losses[-1]
            overall_best_model["losses"] = losses

        filename = f"./model_trained/fnn_model_{i+1}.pt"
        with open(filename, 'w') as f:
            json.dump(result, f)

        print(f"Saved model and hyperparameters to {filename}\n")

    return overall_best_model


batch_size = 1
sequence_length = 4

# Create the DataLoader for first principal component
data = np.array(data)[:, :1000]
print("Data : {} + shape: {} + type: {}".format(data, data.shape, type(data)))
dataloader = create_dataloader(data, batch_size, sequence_length)

# Hyperparameter search space
hyperparams = {
    'hidden_size': [32, 64],
    'learning_rate': [0.01, 0.001],
    'num_epochs': [3000]
}

#Train the model
input_size =  data.shape[0] * sequence_length
output_size = data.shape[0] * 1

#print("Input size : {}".format(input_size))
#train(model, dataloader, num_epochs, learning_rate)
best_model = hyperparameter_training_loop(dataloader, input_size, output_size)

print(f"Best model: {best_model}")
#plot_loss_evolution(best_model["losses"], np.arange(0,best_model["num_epochs"], dtype=int), best_model["learning_rate"], best_model["hidden_size"])




Data : [[ 11.683785    15.96394     18.439566   ... -22.256952   -15.723967
   -5.0476418 ]
 [-15.7863245  -14.464781    -7.952202   ...  14.711998    17.332537
   17.448133  ]
 [ -2.649037     4.804161     4.3103786  ... -13.695016   -17.783976
  -18.565268  ]
 ...
 [ -2.7199492   -1.5206968   -0.4635413  ...   1.8401012    1.5570866
    3.420623  ]
 [  0.39527023   2.331261     2.800027   ...   0.04977342  -2.955885
   -2.7341406 ]
 [  2.1327453   -2.886382     0.02464208 ...  -1.5233226    0.24590252
   -0.606391  ]] + shape: (20, 1000) + type: <class 'numpy.ndarray'>
Testing parameter combination 1/4: (32, 0.01, 3000)
Epoch [1/3000], Loss: 51.331617184432154
Epoch [101/3000], Loss: 42.50839155792711
Epoch [201/3000], Loss: 46.79941790697565
Epoch [301/3000], Loss: 46.65927815437317
Epoch [401/3000], Loss: 46.660426543898375
Epoch [501/3000], Loss: 46.66170070736284
Epoch [601/3000], Loss: 46.662495964502234
